# Model GLM2 - Manipulation Check
### Investigates correlations between 1) regional activation for fixed following conditional trials, and 2) conditional performance

In [ ]:
from os.path import join
from glob import glob
from scipy import stats
from scipy.stats import pearsonr
from pylab import *
from math import sqrt
import nibabel as nb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style("white")
sns.set_palette('muted')

subs = ['WMAZE_001', 'WMAZE_002', 'WMAZE_004', 'WMAZE_005', 'WMAZE_006', 
        'WMAZE_007', 'WMAZE_008', 'WMAZE_009', 'WMAZE_010', 'WMAZE_012', 
        'WMAZE_017', 'WMAZE_018', 'WMAZE_019', 'WMAZE_020', 'WMAZE_021',  
        'WMAZE_022', 'WMAZE_023', 'WMAZE_024', 'WMAZE_026', 'WMAZE_027']
proj_dir = '/home/data/madlab/data/mri/wmaze' 
mask_filenames = []
cope_files = []
for sub in subs:
    mask_filenames_glob = glob(proj_dir + '/roi_analysis/anat_masks/_subject_id_'+ sub +'/_anatmask_xfm*/*')
    mask_filenames.append(sorted(mask_filenames_glob))
    subjcopes_glob = glob(proj_dir +'/scndlvl/model_GLM2/'+ sub +'/fixedfx/cope_*')
    cope_files.append(sorted(subjcopes_glob))
    if len(cope_files[-1]) == 0:
        print(sub)

### Cell to double-check the array indexing

In [ ]:
#change first index of cope_files to indicate participant index in sids array
for i, curr_mask in enumerate(mask_filenames[0]):
    print(i, mask_filenames[0][i].split('/')[-1][:-7]) 

In [ ]:
for i, curr_cope in enumerate(cope_files[0]):
    print(i, cope_files[0][i].split('/')[-1][5:-7]) 

### Use binarized mask to obtain activation in left & right hemisphere for each region of interest

In [ ]:
all_data = {'subjid':[],
            'lhhp_fixed_corr':[], 'rhhp_fixed_corr':[], 
            'lhhp_fixed_incorr':[], 'rhhp_fixed_incorr':[],
            'lhhp_cond_corr':[], 'rhhp_cond_corr':[], 
            'lhhp_cond_incorr':[], 'rhhp_cond_incorr':[],
            'lhcaud_fixed_corr':[], 'rhcaud_fixed_corr':[], 
            'lhcaud_fixed_incorr':[], 'rhcaud_fixed_incorr':[],
            'lhcaud_cond_corr':[], 'rhcaud_cond_corr':[], 
            'lhcaud_cond_incorr':[], 'rhcaud_cond_incorr':[],
            'lhput_fixed_corr':[], 'rhput_fixed_corr':[], 
            'lhput_fixed_incorr':[], 'rhput_fixed_incorr':[],
            'lhput_cond_corr':[], 'rhput_cond_corr':[], 
            'lhput_cond_incorr':[], 'rhput_cond_incorr':[],
            'lhmpfc_fixed_corr':[], 'rhmpfc_fixed_corr':[], 
            'lhmpfc_fixed_incorr':[], 'rhmpfc_fixed_incorr':[],
            'lhmpfc_cond_corr':[], 'rhmpfc_cond_corr':[], 
            'lhmpfc_cond_incorr':[], 'rhmpfc_cond_incorr':[]}

for i in range(len(subs)):
    all_data['subjid'].append(subs[i])
    #ROI masks
    lh_hp_img = nb.load(mask_filenames[i][2])
    rh_hp_img = nb.load(mask_filenames[i][11])
    lh_mpfc_img = nb.load(mask_filenames[i][4])
    rh_mpfc_img = nb.load(mask_filenames[i][13])
    lh_caud_img = nb.load(mask_filenames[i][7])
    rh_caud_img = nb.load(mask_filenames[i][8])
    lh_put_img = nb.load(mask_filenames[i][16])
    rh_put_img = nb.load(mask_filenames[i][17])   
    #copes
    fixed_corr_img = nb.load(cope_files[i][20])
    fixed_incorr_img = nb.load(cope_files[i][21])
    cond_corr_img = nb.load(cope_files[i][14])
    cond_incorr_img = nb.load(cope_files[i][15]) 
    
    region = ['hp', 'mpfc', 'caud', 'put']
    learn_type = ['fixed_corr', 'fixed_incorr', 'cond_corr', 'cond_incorr']   
    for r in region:
        for l in learn_type:
            lh_data = eval('{0}_img.get_data()[lh_{1}_img.get_data() > 0.]'.format(l,r))
            all_data['lh{0}_{1}'.format(r,l)].append(np.mean(lh_data)) 
            rh_data = eval('{0}_img.get_data()[rh_{1}_img.get_data() > 0.]'.format(l,r))
            all_data['rh{0}_{1}'.format(r,l)].append(np.mean(rh_data))
    
all_data_df = pd.DataFrame(all_data)

In [ ]:
region = ['hp', 'mpfc', 'caud', 'put']
for r in region:
    all_data_df['{0}_all_fixed_4'.format(r,l)] = (all_data_df['lh{0}_fixed_corr'.format(r)] 
                                                  + all_data_df['rh{0}_fixed_corr'.format(r)]
                                                  + all_data_df['lh{0}_fixed_incorr'.format(r)] 
                                                  + all_data_df['rh{0}_fixed_incorr'.format(r)])/4.
    all_data_df['{0}_all_cond_4'.format(r,l)] = (all_data_df['lh{0}_fixed_corr'.format(r)] 
                                                 + all_data_df['rh{0}_cond_corr'.format(r)]
                                                 + all_data_df['lh{0}_cond_incorr'.format(r)] 
                                                 + all_data_df['rh{0}_cond_incorr'.format(r)])/4.

# Correlation Scatterplots

In [ ]:
all_prop_corr = []

for sub in subs:
    sub_dir = '/home/data/madlab/data/mri/wmaze/scanner_behav/{0}/'.format(sub)
    dir_file = glob(join(sub_dir, '{0}_wmazebl_2015*.txt'.format(sub)))   
    dir_file.sort()    
    
    data_run1 = pd.read_table(dir_file[0])
    data_run1 = data_run1[:-3]
    data_run2 = pd.read_table(dir_file[1])
    data_run2 = data_run2[:-3]
    data_run3 = pd.read_table(dir_file[2])
    data_run3 = data_run3[:-3]
    data_run4 = pd.read_table(dir_file[3])
    data_run4 = data_run4[:-3]
    data_run5 = pd.read_table(dir_file[4])
    data_run5 = data_run5[:-3]
    data_run6 = pd.read_table(dir_file[5])
    data_run6 = data_run6[:-3]
    
    data_set = pd.concat([data_run1, data_run2, data_run3, data_run4, data_run5, data_run6]) 
    
    trialtype = data_set['TrialType'].values
    correct = data_set['Correct'].values     
    #removing last trial and shifting down 
    trial_shift = trialtype[:-1] 
    trial_shift = np.insert(trial_shift, 0, -1)
    correct_shift = correct[:-1]
    correct_shift = np.insert(correct_shift, 0, -1)
    
    corr = sorted(np.where(((trial_shift == 'B') & (correct == 1)) & (trialtype != 'BL'))[0])    
    incorr = sorted(np.where(((trial_shift == 'B') & (correct == 0)) & (trialtype != 'BL'))[0])   
       
    all_prop_corr.append(len(corr)/float(len(corr) + len(incorr)))
    
all_data_df['prop_corr'] = all_prop_corr

## Hippocampus

In [ ]:
x = all_data_df['prop_corr']
y = all_data_df['hp_all_cond_4']

print 'COND --> FIXED', pearsonr(x,y)
fig, ax = plt.subplots()
ax = sns.regplot(x = 'prop_corr', y = 'hp_all_cond_4', data = all_data_df)
ax.set_ylabel('HPC Conditional Activation')
ax.set_xlabel('Avg Following Fixed Performance')
ax.set_title('Hippocampus')
plt.show()

## Caudate

In [ ]:
x = all_data_df['prop_corr']
y = all_data_df['caud_all_cond_4']

print 'COND --> FIXED', pearsonr(x,y)
fig, ax = plt.subplots()
ax = sns.regplot(x = 'prop_corr', y = 'caud_all_cond_4', data = all_data_df, color = "#f9c402")
ax.set_ylabel('Caudate Conditional Activation')
ax.set_xlabel('Avg Following Fixed Performance')
ax.set_title('Caudate')
plt.show()

## Putamen

In [ ]:
x = all_data_df['prop_corr']
y = all_data_df['put_all_cond_4']

print 'COND --> FIXED', pearsonr(x,y)
fig, ax = plt.subplots()
ax = sns.regplot(x = 'prop_corr', y = 'put_all_cond_4', data = all_data_df, color = "#c34aef")
ax.set_ylabel('Putamen Conditional Activation')
ax.set_xlabel('Avg Following Fixed Performance')
ax.set_title('Putamen')
plt.show()

## Medial PFC

In [ ]:
x = all_data_df['prop_corr']
y = all_data_df['mpfc_all_cond_4']

print 'COND --> FIXED', pearsonr(x,y)
fig, ax = plt.subplots()
ax = sns.regplot(x = 'prop_corr', y = 'mpfc_all_cond_4', data = all_data_df, color = "#fc8b02")
ax.set_ylabel('mPFC Conditional Activation')
ax.set_xlabel('Avg Following Fixed Performance')
ax.set_title('Medial PFC')
plt.show()